In [10]:
%pylab inline

import os
import unicodedata, re
from PIL import Image
from xml.etree import ElementTree as ET
Image.MAX_IMAGE_PIXELS = None

Populating the interactive namespace from numpy and matplotlib


/usr/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning: pylab import has clobbered these variables: ['style', 'f']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +


In [11]:
from pathvalidate import sanitize_filename, sanitize_filepath

In [12]:
# files_folder = "/home/luca/Documenti/planmap/website_data/original/bianca/"


outfolder = "/home/luca/Documenti/planmap/website_data/artworks/"
dataurl = "https://planmap.eu/sites/default/files/artworks/"



In [13]:
def find_all_images(folder):
    import glob
    jpgs = glob.glob(folder + "/*.jpg")
    pngs = glob.glob(folder + "/*.png")

    images = jpgs + pngs
    
    return sort(images)



In [14]:
def resize_image(file, outfile, maxsize=(512, 512), write=True):
    if outfile[-4:] != "png":
        print("renaming extensions to png")
        outfile = outfile[:-3] + "png"    
    if write:
        im = Image.open(file)
        im.thumbnail(maxsize, Image.ANTIALIAS)
        im.save(outfile, "PNG")
        
    return outfile # actual written file

In [15]:
def generate_page_and_images(files_folder, write=False):
    images = find_all_images(files_folder)
    subfolder = files_folder.rsplit('/', 2)[-1]
    print(f"subfolder {subfolder}")
    os.makedirs(outfolder + subfolder, exist_ok=1)

    div = ET.Element("div", attrib={"class":"rtecenter"})
    
    for image in images:
        print(f"image {image}")
        name = os.path.split(image)[-1]
        filename =sanitize_filepath (name).replace(" ", "_")
        filename_thumb = filename[:-4] + ".thumb.jpg"

        outfile = os.path.join(outfolder, subfolder, filename)
        outfile_thumb = os.path.join(outfolder, subfolder, filename_thumb)
        print(outfile)
        resize_image(image, outfile, np.array((2048, 2048)), write=write)
        resize_image(image, outfile_thumb, np.array((2048, 2048))/4, write=write)
        url = os.path.join(dataurl, subfolder, filename)
        url = url[:-3] + "png"
        url_thumb = os.path.join(dataurl, subfolder, filename_thumb)
        url_thumb = url_thumb[:-3] + "png"
        print(url)
        print(url_thumb)

        link = ET.Element("a", attrib={"href": url})
        img = ET.Element("img", attrib={"alt":f"Broken link: {name}", "src": url_thumb, "style": "border-radius:2%;  object-fit: contain; max-height:250px; widht:auto"})
        link.append(img)
        div.append(link)
        
    asstring = ET.tostring(div, encoding='unicode', method='xml')

#     ET.ElementTree(div).write(os.path.join(outfolder, f"{subfolder}.html"))
    
    return asstring
style="height:725px;max-width:500px;width: expression(this.width > 500 ? 500: true);"

In [16]:
out  = {}

for name in ["bianca", "jacopo", "carlotta"]:
    folder = "/home/luca/Documenti/planmap/website_data/original/" + name
    print(f"working in {folder}")
    out[name] = generate_page_and_images(folder, write=False)
    
    

working in /home/luca/Documenti/planmap/website_data/original/bianca
subfolder bianca
image /home/luca/Documenti/planmap/website_data/original/bianca/Presentazione MARIE_inglese.png
/home/luca/Documenti/planmap/website_data/artworks/bianca/Presentazione_MARIE_inglese.png
renaming extensions to png
renaming extensions to png
https://planmap.eu/sites/default/files/artworks/bianca/Presentazione_MARIE_inglese.png
https://planmap.eu/sites/default/files/artworks/bianca/Presentazione_MARIE_inglese.thumb.png
image /home/luca/Documenti/planmap/website_data/original/bianca/Presentazione MAX_inglese.png
/home/luca/Documenti/planmap/website_data/artworks/bianca/Presentazione_MAX_inglese.png
renaming extensions to png
renaming extensions to png
https://planmap.eu/sites/default/files/artworks/bianca/Presentazione_MAX_inglese.png
https://planmap.eu/sites/default/files/artworks/bianca/Presentazione_MAX_inglese.thumb.png
image /home/luca/Documenti/planmap/website_data/original/bianca/page_1.jpg
/home/l

In [17]:
from jinja2 import Template

template = "./artists.tpl"
outfile = template[:-3] + "html"

ll = open(template, 'r').read()

t = Template(ll)
rendered = t.render(out)


with open(outfile, 'w') as f:
    f.write(rendered)

In [18]:
t.render(out)

'    <div>The PLANMAP outreach program based on drawings is carried out by the\n      UNIPD members and sustained by professional artists and the editors of the\n      PLANCK journal. The program will envisage up to 24 illustrations\n      subdivided on the bases of the different scopes and audiences: 9\n      illustrations are dedicated to kids, 9 of them will constitute a Comics\n      series telling planetary geological mapping during the project\n      development; finally 6 of them, that go under the name of Drawing the\n      Future, will be used to populate the web-site and will depict how\n      planetary exploration will be in the near future also thanks to PLANMAP\n      products. Most of them will be delivered also as flyers on whose reverse\n      side will appear sentences recalling the PLANMAP project activities.</div>\n    <div>&nbsp;</div>\n    <h2>Drawing for Kids: Carlotta Montagna</h2>\n    <p class="rtejustify"><a href="https://www.carlottamontagna.it/"><img alt=""\